In [1]:
import torch.nn as nn

class DenoiseAE(nn.Module):
    """基于CNN的自编码器降噪模型"""
    def __init__(self):
        super(DenoiseAE, self).__init__()
        
        # 编码器：压缩特征（输入1×256×256 → 输出32×64×64）
        self.encoder = nn.Sequential(
            # 第一层卷积：1→16通道，尺寸保持256×256（padding=1）
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),  # 激活函数，增强非线性
            nn.MaxPool2d(kernel_size=2),  # 下采样，尺寸→128×128
            
            # 第二层卷积：16→32通道，尺寸保持128×128
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)   # 下采样，尺寸→64×64
        )
        
        # 解码器：恢复图像（输入32×64×64 → 输出1×256×256）
        self.decoder = nn.Sequential(
            # 第一层反卷积：32→16通道，尺寸→128×128（stride=2上采样）
            nn.ConvTranspose2d(
                in_channels=32, out_channels=16, 
                kernel_size=3, stride=2, padding=1, output_padding=1
            ),
            nn.ReLU(inplace=True),
            
            # 第二层反卷积：16→1通道，尺寸→256×256
            nn.ConvTranspose2d(
                in_channels=16, out_channels=1, 
                kernel_size=3, stride=2, padding=1, output_padding=1
            ),
            nn.Sigmoid()  # 输出归一化到0-1，匹配输入范围
        )
    
    def forward(self, x):
        """前向传播：含噪图像→编码→解码→降噪图像"""
        x = self.encoder(x)  # 编码：提取特征
        x = self.decoder(x)  # 解码：恢复图像
        return x